# Exercices en spark dataframes  

Vous avez à repondre à quelques questions concernant des "stock market data", dans ce cas nous aurons affaire aux données boursières de la société Walmart entre 2012-2017 (**chaque ligne représente une journée de trading**). 

Répondez aux questions ci-dessous.

#### Utilisez walmart_stock.csv pour répondre aux questions ci-dessous!

#### démarrez une session Spark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Basics").getOrCreate()

#### Chargez le fichier CSV  Walmart Stock , demander à spark de deduire les types des données .

In [2]:
df = spark.read.csv('walmart_stock.csv',inferSchema=True, header=True)
df

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: int, Adj Close: double]

#### quelles sont les colonnes?

In [3]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### afficher le schéma?

In [4]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### afficher les 5 premières lignes.

In [5]:
df.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [6]:
for elt in df.head(5):
    print(elt, "\n")

Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996) 

Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475) 

Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539) 

Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922) 

Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004) 



#### Utiliser describe() pour explorer le dataset .

In [7]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Questions Bonus!
#### il existe plusieurs nombre après la virgule pour la moyennes et la stddev dans le describe(). formatter les nombre pour afficher seulement 2. faites attention aux typpes de données que le .describe() retourne,  [regardez ce lien](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)



In [12]:
from pyspark.sql.functions import format_number

In [25]:
dfdescribe = df.describe()
dfdescribe.select(dfdescribe['summary'],
                  format_number(dfdescribe['Open'].cast('float'),2).alias('Open'),
                  format_number(dfdescribe['High'].cast('float'),2).alias('High'),
                  format_number(dfdescribe['Low'].cast('float'),2).alias('Low'),
                  format_number(dfdescribe['Close'].cast('float'),2).alias('Close'),
                  format_number(dfdescribe['Volume'].cast('float'),0).alias('Volume'),
                  format_number(dfdescribe['Adj Close'].cast('float'),2).alias('Adj Close')
                 ).show()

+-------+--------+--------+--------+--------+----------+---------+
|summary|    Open|    High|     Low|   Close|    Volume|Adj Close|
+-------+--------+--------+--------+--------+----------+---------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258| 1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,094|    67.24|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781|     6.72|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900|    50.36|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096|    84.91|
+-------+--------+--------+--------+--------+----------+---------+



#### Créez un dataframe avec une colonnes nommée HV Ratio qui contient le ratio (/) entre la colonne High et la colonnes Volume pour chaque journée (ligne) 

In [34]:
df_HV = df.withColumn('HV Ratio',df['High']/df['Volume']).select('HV Ratio')

In [35]:
df_HV.show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### quelle journée avait la valeur du prix High (colonne) la plus élevée ?

In [46]:
df.orderBy(df["High"].desc()).select('Date').head(1)[0]['Date']

datetime.datetime(2015, 1, 13, 0, 0)

#### la moyenne de la colonne close ?

In [47]:
df.agg({'Close':'mean'}).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### le max le min de la colonne volume?

In [53]:
df.agg({'Volume':'max'}).show()

+-----------+
|max(Volume)|
+-----------+
|   80898100|
+-----------+



In [49]:
df.agg({'Volume':'min'}).show()

+-----------+
|min(Volume)|
+-----------+
|    2094900|
+-----------+



## OU

In [54]:
# Ou 
from pyspark.sql.functions import mean

df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [55]:
from pyspark.sql.functions import min, max

df.select(min('Volume'), max('Volume')).show()

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|    2094900|   80898100|
+-----------+-----------+



#### combien de jours la colonnes Close (prix) était inférieure à 60 dollars?


In [60]:
df.filter(df['Close'] < 60).select('Date').count()

81

#### Le pourcentage du temps ou la colonne High était supérieure à 80 dollars ?
#### ça veut dire, (nombre de jours ou High>80)/(le nombre de jours du Dataset)

In [64]:
# Total days in the dataset
totaldays = df.select('Date').count()

In [66]:
# Nombre de jours ou High > 80
high_eighty = df.filter(df['High'] > 80).select('Date').count()

In [68]:
# Le pourcentage du temps ou la colonne High était supérieure à 80 dollars
(high_eighty / totaldays) * 100

9.141494435612083

In [ ]:
# Or just do : df.filter('High > 80').count() * 100/df.count()

#### C'est quoi la corrélation de Pearson entre High et Volume?
#### [cliquer ici](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [71]:
from pyspark.sql.functions import corr

df.corr('High', 'Volume')

-0.3384326061737161

#### le max de High par année?

In [76]:
from pyspark.sql.functions import year

In [92]:
years = df.withColumn("Year",year(df['Date']))

In [95]:
years.groupBy('Year').max()['Year', 'max(High)'].show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### C'est quoi la moyenne de Close (la colonne) pour chaque mois ?
#### d'une autre façcon, c'est quoi la moyenne du prix Close pour  Jan,Fev, Mar, etc... votre résultat doit contenir une valeur (moyenne) pour chaque mois 
le résultat doit avoir une allure semblable à celle la (cliquer sur cette zone pour que le tableau ci-dessous d'affiche corréctment) 
* +-----+-----------------+
* |Month|       avg(Close)|
* +-----+-----------------+
* |    1|71.44801958415842|
* |    2|  71.306804443299|

In [96]:
from pyspark.sql.functions import month

months = df.withColumn("Month",month(df['Date']))

In [100]:
res = months.groupBy('Month').avg()['Month', 'avg(Close)']

In [102]:
res.orderBy(months["Month"]).show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

